# 📄 ResearchPaperQA — Retrieval Demo

This notebook demonstrates the retrieval component of the ResearchPaperQA project.

We focus on:
- Loading a research paper (PDF)
- Chunking and embedding
- FAISS similarity search
- Inspecting retrieved chunks and metadata

The goal is to understand *what the RAG system retrieves* before generation.


In [1]:
from pathlib import Path
from pprint import pprint

from rag_chatbot.loader import load_papers
from rag_chatbot.vectorstore import VectorStoreManager


INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
/Users/malak/Desktop/Projects/ResearchPaperQA/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Path to sample papers
DATA_DIR = Path("../data/sample_papers")

documents = load_papers(str(DATA_DIR))

len(documents)

INFO:rag_chatbot.loader:Loading Probabilistic_operator_learning.pdf...
INFO:rag_chatbot.loader:Successfully loaded 117 chunks from 1 files


117

In [3]:
# Inspect one chunk
doc = documents[0]

print("Content:\n")
print(doc.page_content[:500])

print("\nMetadata:")
pprint(doc.metadata)


Content:

Probabilistic operator learning:
generative modeling and uncertainty quantification
for foundation models of differential equations
Benjamin J. Zhang∗a, Siting Liu∗b, Stanley J. Osherc, and Markos A. Katsoulakisd
aDivision of Applied Mathematics, Brown University
bDepartment of Mathematics, University of California, Riverside
cDepartment of Mathematics, University of California, Los Angeles
dDepartment of Mathematics and Statistics, University of Massachusetts Amherst
September 9, 2025
Abstract


Metadata:
{'arxivid': 'https://arxiv.org/abs/2509.05186v2',
 'author': 'Benjamin J. Zhang; Siting Liu; Stanley J. Osher; Markos A. '
           'Katsoulakis',
 'creationdate': '',
 'creator': 'arXiv GenPDF (tex2pdf:)',
 'doi': 'https://doi.org/10.48550/arXiv.2509.05186',
 'file_path': '../data/sample_papers/Probabilistic_operator_learning.pdf',
 'license': 'http://creativecommons.org/licenses/by/4.0/',
 'page': 0,
 'page_label': '1',
 'producer': 'pikepdf 8.15.1',
 'ptex.fullbanner':

In [ ]:
INDEX_DIR = Path("demo_rag_index")

vectorstore = VectorStoreManager(store_type="faiss")

vectorstore.create_or_load_index(
    documents=documents,
    embedding_provider="sentence-transformers",
    index_path=str(INDEX_DIR),
    force_recreate=True,
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:rag_chatbot.embedder:Sentence Transformer initialized with model: all-MiniLM-L6-v2
INFO:rag_chatbot.vectorstore:Creating new faiss index...
INFO:rag_chatbot.vectorstore:Validated 117 documents for indexing
Batches: 100%|██████████| 4/4 [00:01<00:00,  2.23it/s]
INFO:rag_chatbot.vectorstore:Metadata saved successfully
INFO:rag_chatbot.vectorstore:FAISS index created and saved at demo_rag_index


In [5]:
vectorstore.get_stats()

{'store_type': 'faiss',
 'initialized': True,
 'metadata': {'created_at': '2025-12-18T11:24:43.000870',
  'store_type': 'faiss',
  'num_documents': 117,
  'embedding_provider': 'SentenceTransformerWrapper',
  'total_chars': 102288,
  'avg_doc_length': 874.2564102564103,
  'kwargs': {},
  'sample_source_files': ['Probabilistic_operator_learning.pdf']},
 'index_size': 117}

In [6]:
query = "What is the main contribution of the paper?"

results = vectorstore.search_similar(query, k=5)

len(results)

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]
INFO:rag_chatbot.vectorstore:Found 5 similar documents for query


5

In [7]:
for i, (doc, score) in enumerate(results, start=1):
    print(f"\n--- Result #{i} (score={score:.4f}) ---")
    print(f"Source: {doc.metadata.get('source_file')} | Page: {doc.metadata.get('page')}")
    print(doc.page_content[:400])


--- Result #1 (score=1.6366) ---
Source: Probabilistic_operator_learning.pdf | Page: 10
Pα|{yj,zj}J−1
j=1

·


yj, zj	J−1
j=1

| {z }
Posterior
∝P α(·)P{yj,zj}J−1
j=1 |α =P α(·)| {z }
Prior
J−1Y
j=1
Pyj,zj|α

yj, zj
α

| {z }
Likelihood function
(24)
In the language of in-context learning, the Bayesian posterior describes the ‘context’ informed
by the example condition-QoI pairs. The distribution ofnewpredictions under the posterior dis-
tribution, known as theposterior 

--- Result #2 (score=1.6513) ---
Source: Probabilistic_operator_learning.pdf | Page: 10
.(25)
Notice that the posterior predictive distribution (25) is identical to the ICON conditional measure
(19). Moreover, from Theorem 3.1, we know that ICON approximates the mean of (19), so this
implies that ICONapproximates the mean of the posterior predictive distribution.
The Bayesian perspective, however, requires a prior distributionP α and a likelihood model
Py,z|α to be defineda priori, m

--- Result #3 (scor

In [8]:
queries = [
    "What problem does the paper address?",
    "Which methodology is proposed?",
    "What are the main results?"
]

for q in queries:
    print(f"\n=== Query: {q} ===")
    res = vectorstore.search_similar(q, k=2)
    for doc, score in res:
        print(f"- Page {doc.metadata.get('page')} | score={score:.3f}")


=== Query: What problem does the paper address? ===


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]
INFO:rag_chatbot.vectorstore:Found 2 similar documents for query


- Page 8 | score=1.719
- Page 17 | score=1.720

=== Query: Which methodology is proposed? ===


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.46it/s]
INFO:rag_chatbot.vectorstore:Found 2 similar documents for query


- Page 25 | score=1.635
- Page 14 | score=1.662

=== Query: What are the main results? ===


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.59it/s]
INFO:rag_chatbot.vectorstore:Found 2 similar documents for query


- Page 21 | score=1.646
- Page 23 | score=1.660
